### 🛠️ Initialize Notebook Variables

**Only modify entries under _USER CONFIGURATION_.**

In [ ]:
import utils
from apimtypes import *
from console import print_error, print_ok
from azure_resources import get_infra_rg_name

# ------------------------------
#    USER CONFIGURATION
# ------------------------------

rg_location = 'eastus2'
index       = 1
apim_sku    = APIM_SKU.BASICV2              # Options: 'BASICV2', 'STANDARDV2', 'PREMIUMV2'
deployment  = INFRASTRUCTURE.AFD_APIM_PE    # Options: see supported_infras below
api_prefix  = 'template-'                   # ENTER A PREFIX FOR THE APIS TO REDUCE COLLISION POTENTIAL WITH OTHER SAMPLES
tags        = ['tag1', 'tag2']              # ENTER DESCRIPTIVE TAGS



# ------------------------------
#    SYSTEM CONFIGURATION
# ------------------------------

sample_folder    = '_TEMPLATE'
rg_name          = get_infra_rg_name(deployment, index)
supported_infras = [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.APPGW_APIM, INFRASTRUCTURE.APPGW_APIM_PE, INFRASTRUCTURE.SIMPLE_APIM]
nb_helper        = utils.NotebookHelper(sample_folder, rg_name, rg_location, deployment, supported_infras, index = index, apim_sku = apim_sku)

# Define the APIs and their operations and policies

# API 1
# api1_path = f'{api_prefix}api1'
# api1_get  = GET_APIOperation('This is a GET for API 1')
# api1_post = POST_APIOperation('This is a POST for API 1')
# api1      = API(api1_path, 'API 1', api1_path, 'This is API 1', operations = [api1_get, api1_post], tags = tags)

# API n: <name>
# ...

# APIs Array
# apis: List[API] = [api1, apin]
apis: List[API] = []

print_ok('Notebook initialized')

### 🚀 Deploy Infrastructure and APIs

Creates the bicep deployment into the previously-specified resource group. A bicep parameters, `params.json`, file will be created prior to execution.

In [ ]:
# Build the bicep parameters
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]}
}

# Deploy the sample
output = nb_helper.deploy_sample(bicep_parameters)

if output.success:
    # Extract deployment outputs for testing
    afd_endpoint_url = output.get('fdeSecureUrl', 'Front Door Endpoint URL')    # may be deleted if Front Door is not part of a supported infrastructure
    apim_name        = output.get('apimServiceName', 'APIM Service Name')
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_apis        = output.getJson('apiOutputs', 'APIs')

    print_ok('Deployment completed successfully')
else:
    print_error("Deployment failed!")
    raise SystemExit(1)

### ✅ Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
from apimrequests import ApimRequests
from apimtesting import ApimTesting

# Initialize testing framework
tests = ApimTesting("Template Sample Tests", sample_folder, nb_helper.deployment)

# Example API testing (uncomment and customize as needed)
# Determine endpoints, URLs, etc. prior to test execution
# endpoint_url, request_headers = utils.get_endpoint(deployment, rg_name, apim_gateway_url)

# ********** TEST EXECUTIONS **********

# reqs = ApimRequests(afd_endpoint_url, api_subscription_key)
# output = reqs.singleGet('/', msg = 'Calling API via Azure Front Door. Expect 200.')
# tests.verify('expected_value' in output, True)

tests.print_summary()

print_ok('All done!')